INSTALLATION AND IMPORTING

In [36]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob

In [37]:
nltk.download(['punkt', 'stopwords', 'wordnet'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

CONFIGURATION

In [38]:
STOCK_SYMBOL = "TSLA"  # Change to your target stock
os.makedirs("data/processed", exist_ok=True)

DATA LOADING

In [39]:
def load_data():
    """Load and validate raw data files"""
    # Load files
    tweets_path = f"data/raw/{STOCK_SYMBOL}_tweets.csv"
    prices_path = f"data/raw/{STOCK_SYMBOL}_prices.csv"
    
    # Read data
    tweets_df = pd.read_csv(tweets_path)
    stock_df = pd.read_csv(prices_path, parse_dates=['Date'])

    # Auto-detect columns (case-insensitive)
    date_col = next((c for c in tweets_df.columns if 'date' in c.lower()), None)
    text_col = next((c for c in tweets_df.columns if 'text' in c.lower() or 'tweet' in c.lower()), None)
    
    if not date_col or not text_col:
        print(" Available columns in tweets file:", tweets_df.columns.tolist())
        raise ValueError("Need columns: 'date' and 'text' (or similar names)")

    # Standardize column names
    tweets_df = tweets_df.rename(columns={
        date_col: 'date',
        text_col: 'text'
    })
    tweets_df['date'] = pd.to_datetime(tweets_df['date'])
    
    print(" Data loaded successfully")
    print(f"- Tweets: {len(tweets_df)} rows")
    print(f"- Prices: {len(stock_df)} rows")
    
    return tweets_df, stock_df

# Execute loading
tweets_df, stock_df = load_data()
display(tweets_df.head(2))
display(stock_df.head(2))

 Data loaded successfully
- Tweets: 37422 rows
- Prices: 254 rows


,date,text
0,2022-09-29 23:41:16+00:00,Mainstream media has done an amazing job at br...
1,2022-09-29 23:24:43+00:00,Tesla delivery estimates are at around 364k fr...


,Date,Close,High,Low,Open,Volume
0,NaT,TSLA,TSLA,TSLA,TSLA,TSLA
1,2021-09-30,258.49334716796875,263.0433349609375,258.3333435058594,260.3333435058594,53868000


DATA CLEANING

In [40]:
def clean_text(text):
    """Robust text cleaning with error handling"""
    if pd.isna(text):
        return ""
    text = str(text)
    # Remove URLs, mentions, and special chars
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'@\w+|#\w+', '', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    return text.lower().strip()

# Apply cleaning
tweets_df['cleaned_text'] = tweets_df['text'].apply(clean_text)
print("Text cleaning complete")
display(tweets_df[['text', 'cleaned_text']].head(3))

Text cleaning complete


,text,cleaned_text
0,Mainstream media has done an amazing job at br...,mainstream media has done an amazing job at br...
1,Tesla delivery estimates are at around 364k fr...,tesla delivery estimates are at around k from ...
2,3/ Even if I include 63.0M unvested RSUs as of...,even if i include m unvested rsus as of addit...


SENTIMENT ANALYSIS

In [41]:
# Calculate sentiment polarity
tweets_df['sentiment'] = tweets_df['cleaned_text'].apply(
    lambda x: TextBlob(x).sentiment.polarity
)

# Show distribution
print(" Sentiment scores calculated")
print(tweets_df['sentiment'].describe())
display(tweets_df[['cleaned_text', 'sentiment']].sample(3))

 Sentiment scores calculated
count    37422.00000
mean         0.10290
std          0.24888
min         -1.00000
25%          0.00000
50%          0.00000
75%          0.23000
max          1.00000
Name: sentiment, dtype: float64


,cleaned_text,sentiment
19071,my model just hit miles love it even more t...,0.333333
3219,i believe we are entering a calm before the st...,0.067424
32708,go back on feed search amd nvda jbht etsy tsla...,0.102778


DAILY AGGREGATION

In [42]:
# Group by day
daily_sentiment = tweets_df.groupby(tweets_df['date'].dt.date).agg(
    avg_sentiment=('sentiment', 'mean'),
    tweet_count=('sentiment', 'count')
)

print("Daily sentiment aggregated")
display(daily_sentiment.head())

Daily sentiment aggregated


,avg_sentiment,tweet_count
date,,
2021-09-30,0.130081,90
2021-10-01,0.113493,94
2021-10-02,0.139779,116
2021-10-03,0.142316,61
2021-10-04,0.090901,119


MERGE WITH STOCK DATA

In [43]:
# Merge sentiment with stock prices
merged_df = pd.merge(
    stock_df,
    daily_sentiment,
    left_on=stock_df['Date'].dt.date,
    right_index=True,
    how='left'
).ffill()

# ----- ADD THIS -----
# Convert to numeric (critical fix!)
merged_df[['Open', 'High', 'Low', 'Close', 'Volume']] = merged_df[
    ['Open', 'High', 'Low', 'Close', 'Volume']
].apply(pd.to_numeric, errors='coerce')
# -------------------

# Now safe to calculate
merged_df['next_day_change'] = merged_df['Close'].pct_change().shift(-1)
merged_df.dropna(inplace=True)

print(" Merge successful! Sample:")
display(merged_df.head(3))

 Merge successful! Sample:


,Date,Close,High,Low,Open,Volume,avg_sentiment,tweet_count,next_day_change
1,2021-09-30,258.493347,263.043335,258.333344,260.333344,53868000.0,0.130081,90.0,-0.000335
2,2021-10-01,258.406677,260.260010,254.529999,259.466675,51094200.0,0.113493,94.0,0.008140
3,2021-10-04,260.510010,268.989990,258.706665,265.500000,91449900.0,0.090901,119.0,-0.001203


SAVE PROCESSED DATA

In [44]:
# Save final dataset
merged_df.to_csv(f"data/processed/{STOCK_SYMBOL}_final.csv", index=False)
print(f"""
Processing complete!
Saved to: data/processed/{STOCK_SYMBOL}_final.csv
Final shape: {merged_df.shape}
Date range: {merged_df['Date'].min()} to {merged_df['Date'].max()}
""")


Processing complete!
Saved to: data/processed/TSLA_final.csv
Final shape: (252, 9)
Date range: 2021-09-30 00:00:00 to 2022-09-29 00:00:00



In [46]:
# Cell 1: Verifying input data
print("Tweets date range:", tweets_df['date'].min(), "to", tweets_df['date'].max())
print("Stock date range:", stock_df['Date'].min(), "to", stock_df['Date'].max())

# Cell 2: Checking merge keys
print("\nSample merge keys:")
print("Tweets dates:", tweets_df['date'].dt.date.head(3).values)
print("Stock dates:", stock_df['Date'].dt.date.head(3).values)

Tweets date range: 2021-09-30 01:16:13+00:00 to 2022-09-29 23:41:16+00:00
Stock date range: 2021-09-30 00:00:00 to 2022-09-30 00:00:00

Sample merge keys:
Tweets dates: [datetime.date(2022, 9, 29) datetime.date(2022, 9, 29)
 datetime.date(2022, 9, 29)]
Stock dates: [NaT datetime.date(2021, 9, 30) datetime.date(2021, 10, 1)]
